#  LangChain의 RAG 콤포넌트 
- 문서 임베딩(Embeddings) 

### **학습 목표:**  임베딩 모델과 벡터 데이터베이스를 효과적으로 연동할 수 있다

### **실습 자료**: 

- data/transformer.pdf

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) 문서 로드`

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/transformer.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

PDF 문서 개수: 15


`(4) 텍스트 분할`

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,             # 청크 크기  
    chunk_overlap=200,           # 청크 중 중복되는 부분 크기
    length_function=len,         # 글자 수를 기준으로 분할
    separators=["\n\n", "\n", " ", ""],  # 구분자 - 재귀적으로 순차적으로 적용 
)

# PDF 문서를 텍스트로 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

생성된 텍스트 청크 수: 52
각 청크의 길이: [986, 910, 975, 452, 933, 995, 902, 907, 996, 385, 924, 954, 216, 923, 900, 950, 992, 913, 908, 870, 945, 975, 946, 997, 196, 980, 980, 946, 938, 999, 943, 920, 734, 958, 946, 945, 617, 983, 988, 994, 624, 944, 909, 940, 913, 983, 924, 924, 845, 812, 815, 818]


# 문서 임베딩(Document Embedding)

- 개념: 
    - 텍스트를 벡터(숫자 배열)로 변환하는 과정
    - 문서의 의미적 특성을 수치화하여 컴퓨터가 이해하고 처리할 수 있는 형태로 변환 

- 목적:
    - 텍스트 간 유사도 계산 가능
    - 벡터 데이터베이스 저장 및 검색
    - 의미 기반 문서 검색 구현

- LangChain의 임베딩 모델 종류:
    - OpenAI 임베딩
    - HuggingFace 임베딩 
    - Ollama 임베딩

### 1. **OpenAI**

- LangChain에서 가장 널리 사용되는 임베딩 모델 중 하나

- 주요 특징:
    1. 고품질의 임베딩 생성
    2. 다양한 언어 지원 (다국어 지원)
    3. 일관된 성능
    4. 손쉬운 통합

- 사용시 주의사항:
    1. API 키 설정이 필요 (환경 변수 OPENAI_API_KEY)
    2. API 사용량에 따른 비용 발생
    3. 긴 텍스트는 자동으로 분할되지 않으므로 필요시 TextSplitter를 사용


- 모델별 특징
    ```
    모델                    페이지/달러    MTEB 성능     최대입력
    text-embedding-3-small  62,500       62.3%       8191
    text-embedding-3-large   9,615       64.6%       8191
    text-embedding-ada-002  12,500       61.0%       8191
    ```

- 임베딩 벡터 특성
    1. small: 1536 차원
    1. large: 3072 차원
    1. dimensions 파라미터로 차원 축소 가능

`(1) embedding 모델`

In [5]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 모델 생성
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",  # 사용할 모델 이름
    dimensions=None, # 원하는 임베딩 차원 수를 지정 가능 (기본값: None)
    )

# 임베딩 객체 출력
embeddings_model

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000022C18ECA930>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000022C188FF1A0>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [6]:
# 임베딩 모델의 컨텍스트 길이 확인
embeddings_model.embedding_ctx_length

8191

In [7]:
# 임베딩 모델의 임베딩 차원 확인 - 기본값 (None)
embeddings_model.dimensions

In [8]:
# OpenAIEmbeddings 모델 생성할 때 임베딩 차원을 지정하는 예시
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름
    dimensions=1024, # 원하는 임베딩 차원 수를 지정 가능 (기본값: None)
    )

# 임베딩 모델의 임베딩 차원 확인 
embeddings_model.dimensions

1024

In [9]:
# OpenAIEmbeddings 모델 생성
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-small")

# 임베딩 객체 출력
embeddings_openai

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000022C1887E2A0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000022C1887E960>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

`(2) embed_documents 사용`

In [10]:
# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings_openai = embeddings_openai.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_openai)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_openai[0])}")
print(document_embeddings_openai[0])

임베딩 벡터의 개수: 5
임베딩 벡터의 차원: 1536
[-0.002310403622686863, 0.012099752202630043, -0.002436630427837372, 0.015039036981761456, 0.018447164446115494, -0.04565808176994324, -0.003119608387351036, 0.046956416219472885, -0.018239790573716164, -0.031881313771009445, 0.006847810931503773, 0.006965021602809429, -0.018591422587633133, -0.027138788253068924, 0.004891293589025736, -0.015580009669065475, -0.041546691209077835, 0.010053073056042194, 0.05157271400094032, -0.045874472707509995, -0.01464232336729765, -0.027751890942454338, -0.018753714859485626, -0.022702815011143684, 0.004363845102488995, -0.03981557860970497, 0.05031044781208038, 0.017716851085424423, 0.00028372881934046745, -0.023333948105573654, 0.048976048827171326, -0.015273458324372768, -0.029933813959360123, -0.06159874051809311, 0.02044876292347908, 0.03537960350513458, 0.002585398033261299, -0.006807237863540649, -0.005130222998559475, 0.02477654255926609, 0.00756459916010499, 0.02753550186753273, -0.0025651114992797375, 0.02598

`(3) embed_query 사용`

In [11]:
embedded_query_openai = embeddings_openai.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query_openai)}")
print(embedded_query_openai)

쿼리 임베딩 벡터의 차원: 1536
[-0.02255643531680107, 0.022176697850227356, 0.0004912859876640141, 0.005667589604854584, 0.012189589440822601, -0.04480908066034317, -0.026277868077158928, 0.035809293389320374, -0.002643925603479147, 0.014800287783145905, -0.002100425772368908, 0.0008621239103376865, -0.004865393042564392, -0.07359322160482407, 0.009018776938319206, -0.01542685553431511, -0.05980873107910156, -0.022442514076828957, 0.022423528134822845, -0.0692262351512909, -0.029220838099718094, 0.023239964619278908, -0.036853570491075516, 0.00411778362467885, 0.011145309545099735, -0.0526316836476326, 0.010727598331868649, -3.0946419428801164e-05, -0.010803545825183392, -0.03410046920180321, 0.02529054880142212, -0.018910951912403107, -0.0014999649720266461, -0.05897330865263939, 0.04989757016301155, -0.0034152683801949024, -0.0027602205518633127, -0.0083684753626585, -0.004597202874720097, 0.02280326560139656, -0.011885798536241055, 0.0379737988114357, 0.0032989736646413803, 0.03520170971751213

`(4) 유사도 기반 검색`

In [12]:
from langchain_community.utils.math import cosine_similarity
import numpy as np

# 쿼리와 가장 유사한 문서 찾기 함수
def find_most_similar(
        query: str, 
        doc_embeddings: np.ndarray,
        embeddings_model=OpenAIEmbeddings(model="text-embedding-3-small")
        ) -> tuple[str, float]:
    
    # 쿼리 임베딩: OpenAI 임베딩 사용 
    query_embedding = embeddings_model.embed_query(query)

    # 코사인 유사도 계산
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

    # 가장 유사한 문서 인덱스 찾기
    most_similar_idx = np.argmax(similarities)

    # 가장 유사한 문서와 유사도 반환: 문서, 유사도
    return documents[most_similar_idx], similarities[most_similar_idx]

# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(
        query, 
        document_embeddings_openai, 
        embeddings_model=embeddings_openai
        )
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

쿼리: 인공지능이란 무엇인가요?
가장 유사한 문서: 인공지능은 컴퓨터 과학의 한 분야입니다.
유사도: 0.7116

쿼리: 딥러닝과 머신러닝의 관계는 어떻게 되나요?
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도: 0.6817

쿼리: 컴퓨터가 이미지를 이해하는 방법은?
가장 유사한 문서: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
유사도: 0.7053



### 2. **Huggingface**

- LangChain에서 오픈소스 기반의 대표적인 임베딩 모델

- 주요 특징:
    1. 로컬 환경에서 실행 가능
    2. 다양한 사전학습 모델 지원
    3. 커스텀 모델 학습 및 적용 가능
    4. 무료 사용 가능 (API 비용 없음)

- 사용시 주의사항:
    1. 로컬 컴퓨팅 자원 필요 (CPU/GPU)
    2. 초기 모델 다운로드 시간 소요
    3. 메모리 사용량 고려 필요
    4. transformers 라이브러리 설치 필요

- 대표적인 임베딩 모델:
    ```
    모델                            차원      언어             특징      
    all-MiniLM-L6-v2               384     다국어     빠른 속도, 적은 메모리
    all-mpnet-base-v2              768     다국어     높은 성능, 중간 크기
    multilingual-e5-large         1024     다국어     최고 성능, 큰 메모리
    ```

- 임베딩 벡터 특성:
    1. 모델별로 다양한 차원 제공 (128 ~ 1024)
    2. sentence-transformers 기반 구현
    3. BERT 계열 모델 구조 사용
    4. 코사인 유사도 기반 검색 최적화

`(1) embedding 모델`

- langchain_huggingface 설치 필요

In [14]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings  

# Hugging Face의 임베딩 모델 생성
embeddings_bgem3 = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",          # 사용할 모델 이름 - 알리바바 AI의 BGE-m3 모델 (한국어 성능 우수)
    # model_kwargs={'device': 'cuda'}  # GPU 사용시
    # model_kwargs={'device': 'mps'}   # Mac M1 사용시
)

# 임베딩 객체 출력
embeddings_bgem3

HuggingFaceEmbeddings(model_name='BAAI/bge-m3', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

`(2) embed_documents 사용`

In [15]:
# 문서 임베딩
document_embeddings_bgem3 = embeddings_bgem3.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_bgem3)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_bgem3[0])}")
print(document_embeddings_bgem3[0])

임베딩 벡터의 개수: 5
임베딩 벡터의 차원: 1024
[-0.03941447287797928, 0.008764853700995445, -0.012681580148637295, 0.002453188644722104, -0.008944729343056679, -0.007383656222373247, -0.005377353634685278, -0.00905587524175644, 0.032915227115154266, 0.006045476999133825, -0.027012908831238747, -0.027740875259041786, 0.0004441275668796152, 0.03013652190566063, 0.01724284701049328, 0.017090314999222755, 0.025524891912937164, -0.02185608632862568, -0.011341274715960026, -0.05702260881662369, -0.00030168905504979193, 0.013543072156608105, -0.007450108416378498, 0.018574388697743416, 0.0028946688398718834, 0.00863063707947731, -0.0007445056107826531, -0.028904112055897713, 0.020727837458252907, -0.020500602200627327, 0.008069912903010845, -0.026754185557365417, 0.003963078837841749, -0.016303913667798042, -0.07406217604875565, -0.033650390803813934, -0.023871449753642082, -0.03455008938908577, -0.034785930067300797, 0.005482976790517569, -0.0500335618853569, -0.0028035305440425873, -0.02314690500497818, -0

`(3) embed_query 사용`

In [16]:
embedded_query = embeddings_bgem3.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

쿼리 임베딩 벡터의 차원: 1024
[-0.037039078772068024, -0.004837993066757917, 0.0029373057186603546, -0.01551460474729538, -0.0009441875154152513, -0.04150162264704704, -0.006574427708983421, 0.011289620772004128, 0.021614043042063713, 0.004928688518702984, -0.020340582355856895, 0.01690519228577614, -0.012874167412519455, 0.005518920253962278, 0.014988347887992859, 0.024228783324360847, 0.007369132712483406, -0.0280498955398798, -0.01493898592889309, -0.05185187980532646, -0.006705091334879398, -0.009251533076167107, -0.016980808228254318, 0.006491529289633036, 0.052931759506464005, 0.04813732951879501, -0.008069545961916447, -0.023171771317720413, 0.01814304105937481, -0.011328116059303284, -0.004240398295223713, -0.0063547175377607346, -0.002271725330501795, 0.014329456724226475, -0.03563683107495308, -0.0081558208912611, -0.011798232793807983, -0.04542405903339386, -0.040732868015766144, 0.0022138976491987705, -0.01213227491825819, 0.01789613626897335, -0.019144676625728607, -0.04192442446947

`(4) 유사도 기반 검색`

In [17]:
# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(query, document_embeddings_bgem3, embeddings_model=embeddings_bgem3) 
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

쿼리: 인공지능이란 무엇인가요?
가장 유사한 문서: 인공지능은 컴퓨터 과학의 한 분야입니다.
유사도: 0.7269

쿼리: 딥러닝과 머신러닝의 관계는 어떻게 되나요?
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도: 0.7057

쿼리: 컴퓨터가 이미지를 이해하는 방법은?
가장 유사한 문서: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
유사도: 0.6843



### 3. **Ollama**

- LangChain에서 로컬 실행에 최적화된 경량 임베딩 모델

- 주요 특징:
    1. 완전한 로컬 실행 환경 제공
    2. 빠른 추론 속도
    3. 간단한 설치 및 실행 과정
    4. Docker 기반 손쉬운 배포

- 사용시 주의사항:
    1. Ollama 서버 실행 필요
    2. 모델별 시스템 요구사항 확인
    3. API 엔드포인트 설정 필요
    4. 동시 요청 처리량 고려

- 대표적인 임베딩 모델:
    ```
    모델                차원       언어        특징
    llama2              4096      영어       범용성 높은 기본 모델
    nomic-embed-text    768       영어       경량화된 고성능 모델
    codellama          2048       다국어     코드 특화 임베딩
    ```

- 임베딩 벡터 특성:
    1. 모델별 고정 차원 사용
    2. 최적화된 양자화 지원
    3. 배치 처리 최적화

`(1) embedding 모델`

- langchain_ollama 설치 필요

In [18]:
from langchain_ollama import OllamaEmbeddings 

#OllamaEmbeddings 모델 생성
embeddings_ollama = OllamaEmbeddings(
    model="nomic-embed-text",          # 사용할 모델 이름
    base_url="http://localhost:11434"  # Ollama 서버 주소
)
embeddings_ollama = OllamaEmbeddings(model="bge-m3")

# 임베딩 객체 출력
embeddings_ollama

OllamaEmbeddings(model='bge-m3', validate_model_on_init=False, base_url=None, client_kwargs={}, async_client_kwargs={}, sync_client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

`(2) embed_documents 사용`

In [19]:
# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings_ollama = embeddings_model.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_ollama)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_ollama[0])}")
print(document_embeddings_ollama[0])

임베딩 벡터의 개수: 5
임베딩 벡터의 차원: 1024
[-0.0026579881086945534, 0.013920078054070473, -0.0028032048139721155, 0.01730155758559704, 0.021222414448857307, -0.05252702906727791, -0.0035889321006834507, 0.05402068793773651, -0.020983843132853508, -0.03667763993144035, 0.007878017611801624, 0.008012861013412476, -0.021388376131653786, -0.03122163563966751, 0.005627155303955078, -0.01792391575872898, -0.047797106206417084, 0.011565489694476128, 0.05933148041367531, -0.052775971591472626, -0.016845161095261574, -0.03192697465419769, -0.0215750839561224, -0.02611829712986946, 0.0050203558057546616, -0.045805562287569046, 0.057879310101270676, 0.02038222923874855, 0.000326413894072175, -0.026844382286071777, 0.056344158947467804, -0.01757124625146389, -0.0344371534883976, -0.07086584717035294, 0.023525139316916466, 0.040702223777770996, 0.0029743534978479147, -0.007831340655684471, -0.005902030039578676, 0.028504004701972008, 0.008702641353011131, 0.031678032130002975, -0.002951015019789338, 0.02989393

`(3) embed_query 사용`

In [20]:
embedded_query = embeddings_ollama.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

쿼리 임베딩 벡터의 차원: 1024
[-0.03681023, -0.004903051, 0.0029050948, -0.01553997, -0.0009213159, -0.041663706, -0.006701716, 0.011330772, 0.021562899, 0.004902583, -0.020480406, 0.016806316, -0.012873897, 0.0054373853, 0.014988197, 0.02431702, 0.00742082, -0.02795675, -0.015137621, -0.051823303, -0.006601279, -0.009193479, -0.017144404, 0.006588784, 0.052906107, 0.048063856, -0.008165826, -0.023190044, 0.018099017, -0.011383665, -0.0043358565, -0.006360557, -0.00243946, 0.014362668, -0.035457585, -0.00821874, -0.011810058, -0.045279074, -0.04064702, 0.0021692775, -0.012288583, 0.017699474, -0.0191562, -0.04193752, 0.0009997609, -0.039017037, -0.024434805, -0.024545293, -0.022177339, -0.0044823955, 0.03154298, -0.048746303, 0.01799886, 0.02526545, 0.002365806, 0.04854048, -0.008117381, 0.028092192, -0.07375984, -0.020707084, -0.026365895, -0.007490066, -0.038779262, -0.01738154, 0.019240336, 0.06372398, 0.020389654, -0.011854772, -0.018534634, -0.04077229, 0.0025231682, 0.04150272, -0.05778026

`(4) 유사도 기반 검색`

In [21]:
# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(query, document_embeddings_ollama, embeddings_model=embeddings_ollama) 
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

쿼리: 인공지능이란 무엇인가요?
가장 유사한 문서: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
유사도: 0.0528

쿼리: 딥러닝과 머신러닝의 관계는 어떻게 되나요?
가장 유사한 문서: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
유사도: 0.0293

쿼리: 컴퓨터가 이미지를 이해하는 방법은?
가장 유사한 문서: 인공지능은 컴퓨터 과학의 한 분야입니다.
유사도: 0.0154



# [실습 프로젝트]

1. OpenAI text-embedding-3-small 임베딩 모델을 초기화합니다. 
2. 임베딩 차원을 각각 512와 1536으로 구분하여 2개의 모델을 생성합니다. 
3. 아래 주어진 문장들의 임베딩을 생성합니다. (임베딩 차원이 512, 1536인 경우 각각 2개씩 생성)
   - 문장1: "인공지능은 현대 사회를 변화시키고 있다"
   - 문장2: "AI 기술이 우리의 미래를 바꾸고 있다"
4. 생성된 임베딩의 차원을 출력합니다. (임베딩 차원이 512, 1536인 경우를 각각 출력)
5. 두 문장 간의 코사인 유사도를 계산합니다. (임베딩 차원이 512, 1536인 경우를 각각 비교)

In [22]:

# OpenAI text-embedding-3-small 임베딩 모델 초기화 및 비교 (512, 1536)
from langchain_openai import OpenAIEmbeddings
import numpy as np

sentences = [
    "인공지능은 현대 사회를 변화시키고 있다",
    "AI 기술이 우리의 미래를 바꾸고 있다"
]

# 모델 생성: dimensions=512, dimensions=1536
emb_model_512 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512)
emb_model_1536 = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536)

# 임베딩 생성
embs_512 = emb_model_512.embed_documents(sentences)
embs_1536 = emb_model_1536.embed_documents(sentences)

# 차원 출력
print(f"512-d 모델 설정된 차원: {emb_model_512.dimensions}")
print(f"1536-d 모델 설정된 차원: {emb_model_1536.dimensions}")
print(f"512-d 임베딩: 개수={len(embs_512)}, 차원={len(embs_512[0])}")
print(f"1536-d 임베딩: 개수={len(embs_1536)}, 차원={len(embs_1536[0])}")

# 코사인 유사도 계산 함수
def cosine_sim(a, b):
    a = np.array(a, dtype=float)
    b = np.array(b, dtype=float)
    na = np.linalg.norm(a)
    nb = np.linalg.norm(b)
    if na == 0 or nb == 0:
        return 0.0
    return float(np.dot(a, b) / (na * nb))

# 두 문장 간 유사도 계산
sim_512 = cosine_sim(embs_512[0], embs_512[1])
sim_1536 = cosine_sim(embs_1536[0], embs_1536[1])

print(f"코사인 유사도 (512-d): {sim_512:.6f}")
print(f"코사인 유사도 (1536-d): {sim_1536:.6f}")



512-d 모델 설정된 차원: 512
1536-d 모델 설정된 차원: 1536
512-d 임베딩: 개수=2, 차원=512
1536-d 임베딩: 개수=2, 차원=1536
코사인 유사도 (512-d): 0.553863
코사인 유사도 (1536-d): 0.495556
